## Book Copurchase Graph Analysis

#### Table of Contents

* Introduction
* Data Import and Formatting
* Data Exploration
* Graph Processing and Analysis

### Imports

In [1]:
import cudf
import cugraph
import pandas as pd

### Introduction to Dataset

Dataset is the processed version of Amazon Product co-purchasing network metadata taken from SNAP http://snap.stanford.edu/data/amazon-meta.html. 
The original dataset includes about 548,552 different products (Books, music CDs, DVDs, and VHS video tapes)
The dataset used below is the filtered version of the above data which includes only Book Product.

### Load and Explore Dataset

In [2]:
dataset_path = '../data/amazon/books/amazon-books.csv'

In [3]:
%%time
gdf = cudf.DataFrame()
gdf = cudf.read_csv(dataset_path)

CPU times: user 439 ms, sys: 456 ms, total: 895 ms
Wall time: 1.26 s


### Exploration

Let's see how the data looks like. 
ASIN is the identifier for Amazon Book. Copurchased column contains a list of books that are commmonly purchased along with the book in ASIN column. These Copurchased data are collected from the Amazon Recommendation section "Customers who bought this item also bought".

In [4]:
%%time
gdf.head().to_pandas()

CPU times: user 190 ms, sys: 25.3 ms, total: 215 ms
Wall time: 214 ms


,Id,ASIN,Title,Categories,Group,Copurchased,SalesRank,TotalReviews,AvgRating
0,1,0827229534,Patterns of Preaching: A Sermon Sampler,subjects religion preaching clergy spiritualit...,Book,0804215715 156101074X 0687023955 0687074231 08...,396585,2,2.0
1,2,0738700797,Candlemas: Feast of Flames,subjects witchcraft earth religion based spiri...,Book,0738700827 1567184960 1567182836 0738700525 07...,168596,12,12.0
2,3,0486287785,World War II Allied Fighter Planes Trading Cards,general subjects hobbies home garden crafts books,Book,None,1270652,1,1.0
3,4,0842328327,Life Application Bible Commentary: 1 and 2 Tim...,subjects life bibles christian general history...,Book,0842328130 0842330313 0842328610 0842328572,631289,1,1.0
4,5,1577943082,Prayers That Avail Much for Business: Executive,subjects religion prayerbooks devotion worship...,Book,157794349X 0892749504 1577941829 0892749563,455160,0,0.0


Notice that some of the copurchased data for the respective books are missing ("None"). Let's find out how many of them are missing.

In [5]:
# dataset contains 392966 rows.
%time gdf.shape

CPU times: user 4 µs, sys: 4 µs, total: 8 µs
Wall time: 11.7 µs


(392966, 9)

In [6]:
# And all the books are unique
%time gdf.ASIN.unique().shape[0]

CPU times: user 11 ms, sys: 19.1 ms, total: 30 ms
Wall time: 30.2 ms


392966

In [7]:
%%time
# Fill None with empty string for Object Type Columns.
gdf['Copurchased'] = gdf['Copurchased'].fillna('')
gdf['Categories'] = gdf['Categories'].fillna('')

CPU times: user 20.3 ms, sys: 8.23 ms, total: 28.5 ms
Wall time: 80 ms


In [8]:
# There are 126553 records, 32% of the data are missing Copurchased info.
%time gdf[gdf.Copurchased == ''].shape[0] / gdf.shape[0]

CPU times: user 883 ms, sys: 245 ms, total: 1.13 s
Wall time: 1.2 s


0.32204567316256366

These missing Copurchased records are not gonna help us in finding which books are most similar to them. So we will reduce the number of book records.

In [9]:
%time reduced_gdf = gdf[gdf.Copurchased != '']

CPU times: user 59.4 ms, sys: 139 ms, total: 198 ms
Wall time: 295 ms


In [10]:
%time reduced_gdf.shape[0]

CPU times: user 4 µs, sys: 4 µs, total: 8 µs
Wall time: 10.5 µs


266413

### Preprocessing for Graph Edgelist

Convert cudf to Pandas to use the Pandas APIs needed to split (explode) a column into multiple rows

In [11]:
%time pd_df = reduced_gdf.to_pandas()

CPU times: user 288 ms, sys: 179 ms, total: 467 ms
Wall time: 577 ms


In [12]:
%time pd_df.head(2)

CPU times: user 1.62 ms, sys: 0 ns, total: 1.62 ms
Wall time: 1.13 ms


,Id,ASIN,Title,Categories,Group,Copurchased,SalesRank,TotalReviews,AvgRating
0,1,0827229534,Patterns of Preaching: A Sermon Sampler,subjects religion preaching clergy spiritualit...,Book,0804215715 156101074X 0687023955 0687074231 08...,396585,2,2.0
1,2,0738700797,Candlemas: Feast of Flames,subjects witchcraft earth religion based spiri...,Book,0738700827 1567184960 1567182836 0738700525 07...,168596,12,12.0


Create new dataframe, splitting books in Copurchased into individual rows with ASIN as the index

In [13]:
%time new_pd_df = pd.DataFrame(pd_df.Copurchased.str.split(' ').tolist(), index=pd_df.ASIN).stack()

CPU times: user 799 ms, sys: 51.6 ms, total: 850 ms
Wall time: 849 ms


In [14]:
%time new_pd_df.head(6)

CPU times: user 385 µs, sys: 0 ns, total: 385 µs
Wall time: 390 µs


ASIN         
0827229534  0    0804215715
            1    156101074X
            2    0687023955
            3    0687074231
            4    082721619X
0738700797  0    0738700827
dtype: object

In [15]:
# get rid of secondary index
# make ASIN as a column (it can't be an index since the values will be duplicate)
%time new_pd_df = new_pd_df.reset_index([0, 'ASIN'])

CPU times: user 35 ms, sys: 0 ns, total: 35 ms
Wall time: 34.5 ms


In [16]:
%time new_pd_df.head(2)

CPU times: user 404 µs, sys: 238 µs, total: 642 µs
Wall time: 538 µs


,ASIN,0
0,0827229534,0804215715
1,0827229534,156101074X


In [17]:
# to save memory, select only the columns we need for our graph
# rename column '0' to column 'Copurchase_ASIN'
%time new_pd_df.columns = ['ASIN', 'Copurchased_ASIN']

CPU times: user 336 µs, sys: 198 µs, total: 534 µs
Wall time: 387 µs


In [18]:
%time new_pd_df.head(2)

CPU times: user 512 µs, sys: 0 ns, total: 512 µs
Wall time: 453 µs


,ASIN,Copurchased_ASIN
0,0827229534,0804215715
1,0827229534,156101074X


In [19]:
%%time
new_gdf = cudf.from_pandas(new_pd_df)

CPU times: user 97.6 ms, sys: 65 ms, total: 163 ms
Wall time: 169 ms


In [20]:
%%time
# Merge exploded rows of Copurchased into original dataset.
combined_gdf = cudf.merge(new_gdf, gdf, on=['ASIN']).sort_values(['ASIN'])

CPU times: user 296 ms, sys: 371 ms, total: 667 ms
Wall time: 868 ms


In [21]:
%%time
combined_gdf.head(3).to_pandas()

CPU times: user 20.2 ms, sys: 0 ns, total: 20.2 ms
Wall time: 19.1 ms


,ASIN,Copurchased_ASIN,Id,Title,Categories,Group,Copurchased,SalesRank,TotalReviews,AvgRating
432481,0001047655,0061099341,271961,Prodigal Daughter,general tape subjects literature contemporary ...,Book,0061007129 0061007358 0061007137 0061099341 00...,1116690,30,30.0
432485,0001047655,0061007161,271961,Prodigal Daughter,general tape subjects literature contemporary ...,Book,0061007129 0061007358 0061007137 0061099341 00...,1116690,30,30.0
432502,0001047655,0061007129,271961,Prodigal Daughter,general tape subjects literature contemporary ...,Book,0061007129 0061007358 0061007137 0061099341 00...,1116690,30,30.0


Remove Copurchased Columns which is redundant.

In [22]:
%%time
combined_gdf = combined_gdf.drop('Copurchased')
combined_gdf = combined_gdf.drop('Group')

CPU times: user 16 ms, sys: 20 ms, total: 36.1 ms
Wall time: 35.1 ms


### Unipartite Correlation Network

We will form a Unipartite Graph between ASIN and Copurchased_ASIN. Based on the cuGraph requirements, we have to convert the aforementioned columns' Object Datatype to int32 type and vertex ids have to start from 0. So we will be creating columns of renumbered source vertex ids and destination vertex ids, both will be in int32 type. The numbering map from renumbering will map the new ids to original ids. The current renumbering API from cuGraph only support int32 type.

In [23]:
%%time
combined_gdf.dtypes

CPU times: user 306 µs, sys: 194 µs, total: 500 µs
Wall time: 506 µs


ASIN                 object
Copurchased_ASIN     object
Id                    int64
Title                object
Categories           object
SalesRank             int64
TotalReviews          int64
AvgRating           float64
dtype: object

In [24]:
%%time
combined_gdf.add_column('ASIN_int', combined_gdf['ASIN'].astype('int32'))
combined_gdf.add_column('Copurchased_ASIN_int', combined_gdf['Copurchased_ASIN'].astype('int32'))

CPU times: user 8.17 ms, sys: 82 µs, total: 8.25 ms
Wall time: 43.6 ms


In [25]:
%%time
G = cugraph.Graph()

# The cugraph renumbering feature allows us to take two columns of any integer type and translate them 
# into a densely packed contiguous array numbered from 0 to (num_unique_values - 1).
# These renumbered vertices can be used to create a graph much more efficiently.

src_r, dst_r, numbering = G.renumber(combined_gdf['ASIN_int'], combined_gdf['Copurchased_ASIN_int'])

CPU times: user 7.01 ms, sys: 2.78 ms, total: 9.79 ms
Wall time: 14.5 ms


In [26]:
%%time
combined_gdf.add_column("src_id", src_r)
combined_gdf.add_column("dst_id", dst_r)

CPU times: user 78 µs, sys: 50 µs, total: 128 µs
Wall time: 133 µs


In [27]:
numbering.head(4).to_pandas()

0     60621591
1     62505521
2     76790625
3    152008578
dtype: int32

Explore by querying an original dataframe for book title or ASIN.


In [28]:
%%time
query_range = combined_gdf[combined_gdf.SalesRank > 0]

CPU times: user 193 ms, sys: 244 ms, total: 438 ms
Wall time: 651 ms


In [29]:
query_range[query_range.SalesRank < 200].sort_values('SalesRank', ascending=True).to_pandas()

,ASIN,Copurchased_ASIN,Id,Title,Categories,SalesRank,TotalReviews,AvgRating,ASIN_int,Copurchased_ASIN_int,src_id,dst_id
42690,0385504209,0440136482,296,The Da Vinci Code,general authors mystery subjects dan book spec...,19,3049,3049.0,385504209,440136482,97478,41363
42712,0385504209,0671027360,296,The Da Vinci Code,general authors mystery subjects dan book spec...,19,3049,3049.0,385504209,671027360,97478,140367
42716,0385504209,0671027387,296,The Da Vinci Code,general authors mystery subjects dan book spec...,19,3049,3049.0,385504209,671027387,97478,141277
686839,0385730586,0385729340,390452,Sisterhood of the Traveling Pants (Sisterhood ...,children social subjects literature general si...,21,746,746.0,385730586,385729340,409,229112
686843,0385730586,037582233X,390452,Sisterhood of the Traveling Pants (Sisterhood ...,children social subjects literature general si...,21,746,746.0,385730586,37582233,409,61985
686847,0385730586,0609807900,390452,Sisterhood of the Traveling Pants (Sisterhood ...,children social subjects literature general si...,21,746,746.0,385730586,609807900,409,142910
197515,0316346624,0066620996,89000,The Tipping Point: How Little Things Can Make ...,subjects office com general home psychology in...,23,437,437.0,316346624,66620996,64862,118003
549475,0142001740,0156027321,337971,The Secret Life of Bees,general subjects literature contemporary ficti...,26,996,996.0,142001740,156027321,83262,170028
227168,0066620996,0060516402,154855,Good to Great: Why Some Companies Make the Lea...,planning subjects primers office competition c...,29,363,363.0,66620996,60516402,118003,41514
227172,0066620996,0609610570,154855,Good to Great: Why Some Companies Make the Lea...,planning subjects primers office competition c...,29,363,363.0,66620996,609610570,118003,117789


Create a Directed Graph of a copurchase network. Edges are pointing from one book to another. 

In [30]:
G.add_edge_list(combined_gdf["src_id"], combined_gdf["dst_id"])

In [31]:
G.number_of_edges()

910848

In [32]:
G.number_of_vertices()

270038

In [33]:
degree = G.degree()
in_degree = G.in_degree()
out_degree = G.out_degree()

We get the edge lists for the two-hop neighbors. (traverse two hops, starting node is first and after two hops, the end node is second) start->hop->end

In [34]:
two_hop = G.get_two_hop_neighbors()
two_hop

<cudf.DataFrame ncols=2 nrows=2170614 >

In [35]:
two_hop[two_hop.first==0].to_pandas()

,first,second
0,0,9307
1,0,21939
2,0,106041
3,0,146031
4,0,150209
5,0,234029


In [36]:
combined_gdf[combined_gdf.ASIN == '0890420254'].to_pandas()

,ASIN,Copurchased_ASIN,Id,Title,Categories,SalesRank,TotalReviews,AvgRating,ASIN_int,Copurchased_ASIN_int,src_id,dst_id
752320,0890420254,0898625688,458358,Diagnostic and Statistical Manual of Mental Di...,body reference general medicine subjects medic...,371,35,35.0,890420254,898625688,39084,245652
752324,0890420254,0890420262,458358,Diagnostic and Statistical Manual of Mental Di...,body reference general medicine subjects medic...,371,35,35.0,890420254,890420262,39084,39364
753495,0890420254,0890420270,458358,Diagnostic and Statistical Manual of Mental Di...,body reference general medicine subjects medic...,371,35,35.0,890420254,890420270,39084,39625
753499,0890420254,0963382136,458358,Diagnostic and Statistical Manual of Mental Di...,body reference general medicine subjects medic...,371,35,35.0,890420254,963382136,39084,193262
753503,0890420254,1585620599,458358,Diagnostic and Statistical Manual of Mental Di...,body reference general medicine subjects medic...,371,35,35.0,890420254,1585620599,39084,224448


In [37]:
ASIN = '0890420254'
ASIN_int = 890420254
book_1 = numbering[numbering == ASIN_int].index[0]
book_1

39084

In [38]:
%%time
clist = cudf.DataFrame()
clist["second"] = two_hop.second.unique().astype("int32")

CPU times: user 217 ms, sys: 27.8 ms, total: 245 ms
Wall time: 244 ms


In [39]:
%%time
clist["first"] = book_1.astype("int32")

CPU times: user 159 ms, sys: 6.85 ms, total: 166 ms
Wall time: 164 ms


In [40]:
%%time
degree_query = degree[degree.vertex == book_1]
in_degree_query = in_degree[in_degree.vertex == book_1]
out_degree_query = out_degree[out_degree.vertex == book_1]

CPU times: user 46.8 ms, sys: 36.3 ms, total: 83.1 ms
Wall time: 146 ms


In [41]:
degree_query.to_pandas()

,vertex,degree
39084,39084,329


book_1 has more incoming edges than outgoing ones. That means through the purchase of connected neighbor books, people discover book_1 and purchased it, and that's why the Sales Ranking for book_1 is high as well.

In [42]:
in_degree_query.to_pandas()

,vertex,degree
39084,39084,324


In [43]:
out_degree_query.to_pandas()

,vertex,degree
39084,39084,5


The Jaccard similarity between two sets is defined as the ratio of the volume of their intersection divided by the volume of their union. 

The Jaccard Similarity can then be defined as

<a href="https://www.codecogs.com/eqnedit.php?latex=js(A,B)&space;=&space;\frac{|A&space;\cap&space;B|}{|A&space;\cup&space;B&space;|&space;}&space;=&space;\frac{|A&space;\cap&space;B|}{&space;|A|&space;&plus;&space;|B|&space;-&space;|A&space;\cup&space;B&space;|&space;}" target="_blank"><img src="https://latex.codecogs.com/gif.latex?js(A,B)&space;=&space;\frac{|A&space;\cap&space;B|}{|A&space;\cup&space;B&space;|&space;}&space;=&space;\frac{|A&space;\cap&space;B|}{&space;|A|&space;&plus;&space;|B|&space;-&space;|A&space;\cup&space;B&space;|&space;}" title="js(A,B) = \frac{|A \cap B|}{|A \cup B | } = \frac{|A \cap B|}{ |A| + |B| - |A \cup B | }" /></a>


cuGraph's Jaccard computation allows us to pass a list of src/dst vertices to compute, so that we don't have to compute all possible combinations. 
We try to find what are the other books similar to book_1 using Jaccard Similarity.

In [44]:
%%time
jacc_clist_1 = cugraph.jaccard(G, first=clist.first, second=clist.second)

CPU times: user 116 ms, sys: 3.51 ms, total: 120 ms
Wall time: 118 ms


In [45]:
%%time
jacc_clist_1.sort_values("jaccard_coeff", ascending=False).head(25).to_pandas()

CPU times: user 19.3 ms, sys: 7.5 ms, total: 26.8 ms
Wall time: 81.3 ms


,source,destination,jaccard_coeff
20927,39084,39084,1.000000
21068,39084,39364,0.428571
21213,39084,39625,0.428571
118356,39084,220121,0.428571
106142,39084,197246,0.333333
76603,39084,142630,0.285714
119188,39084,221679,0.285714
132283,39084,245951,0.285714
11607,39084,21752,0.250000
103976,39084,193262,0.250000


In [46]:
%%time
combined_gdf[combined_gdf.ASIN_int == numbering[book_1]].to_pandas()

CPU times: user 26.1 ms, sys: 53.6 ms, total: 79.8 ms
Wall time: 146 ms


,ASIN,Copurchased_ASIN,Id,Title,Categories,SalesRank,TotalReviews,AvgRating,ASIN_int,Copurchased_ASIN_int,src_id,dst_id
752320,0890420254,0898625688,458358,Diagnostic and Statistical Manual of Mental Di...,body reference general medicine subjects medic...,371,35,35.0,890420254,898625688,39084,245652
752324,0890420254,0890420262,458358,Diagnostic and Statistical Manual of Mental Di...,body reference general medicine subjects medic...,371,35,35.0,890420254,890420262,39084,39364
753495,0890420254,0890420270,458358,Diagnostic and Statistical Manual of Mental Di...,body reference general medicine subjects medic...,371,35,35.0,890420254,890420270,39084,39625
753499,0890420254,0963382136,458358,Diagnostic and Statistical Manual of Mental Di...,body reference general medicine subjects medic...,371,35,35.0,890420254,963382136,39084,193262
753503,0890420254,1585620599,458358,Diagnostic and Statistical Manual of Mental Di...,body reference general medicine subjects medic...,371,35,35.0,890420254,1585620599,39084,224448
